# Defining MDM

In [1]:
def mdm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="RMSE"):
    def error_check():
        rt = 0
        msg = ""
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)

        if (crit != "RMSE" and crit != "MAE" and crit != "MAPE"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    

    error_code = error_check()

    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return

    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    T = float(len(actual_lst))
    
    if crit == "RMSE":
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append((actual - p1) ** 2)
            e2_lst.append((actual - p2) ** 2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    if (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
   
    mean_d = pd.Series(d_lst).mean()
    
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    MDM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    MDM_stat = harvey_adj*MDM_stat
    p_value = 2*t.cdf(-abs(MDM_stat), df = T - 1)
    
    dm_return = collections.namedtuple('mdm_return', 'MDM p_value')
    
    rt = dm_return(MDM = MDM_stat, p_value = p_value)
    
    return rt

# Importing real values

In [2]:
import random
import numpy as np
import pandas as pd
from pandas import read_csv
actual_lst = read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,22])
actual_lst=actual_lst.values
actual_lst=actual_lst.flatten()
actual_lst

array([3600.160021, 3588.679729, 3567.330034, 3543.84029 , 3556.850198,
       3552.770191, 3594.51992 , 3617.760102, 3629.33006 , 3594.39006 ,
       3645.870055, 3644.840004, 3657.170132, 3670.939769, 3678.879815,
       3678.829902, 3660.539881, 3645.180115, 3633.399907, 3645.840019,
       3659.620257, 3688.569886, 3710.870125, 3685.840164, 3636.4799  ,
       3676.15981 , 3689.280144, 3689.319897, 3723.029755, 3723.309795,
       3730.209631, 3726.879635, 3662.709966, 3695.069981, 3705.340008,
       3764.709695, 3783.59999 , 3789.020122, 3776.50978 , 3791.499841,
       3792.859843, 3749.61981 , 3780.369819, 3816.219726, 3845.050096,
       3830.409864, 3797.159818, 3847.779818, 3732.479982, 3755.749758,
       3694.120321, 3725.619899, 3791.839952, 3816.67998 , 3836.659955,
       3874.929951, 3892.590104, 3902.639722, 3884.939815, 3890.389984,
       3905.779785, 3923.849838, 3900.429884, 3885.029923, 3903.06994 ,
       3874.709983, 3805.590153, 3859.599779, 3814.039924, 3789.

# Import the predictions of the proposed model

In [3]:
pred1_lst =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,23])
pred1_lst =pred1_lst.values
pred1_lst=pred1_lst.flatten()
pred1_lst

array([3582.5515, 3580.2136, 3555.7017, 3547.4583, 3548.938 , 3561.7751,
       3585.3691, 3611.0789, 3616.3137, 3607.4033, 3623.6765, 3647.6177,
       3650.3176, 3662.6052, 3673.8503, 3667.7554, 3657.7727, 3643.5886,
       3634.647 , 3641.7031, 3657.371 , 3678.74  , 3702.9192, 3683.9326,
       3650.1501, 3669.1963, 3687.1409, 3698.806 , 3715.7837, 3718.646 ,
       3731.7803, 3724.6074, 3681.4456, 3689.1985, 3713.7666, 3749.3477,
       3788.0732, 3777.929 , 3775.631 , 3799.698 , 3790.0159, 3766.7231,
       3777.096 , 3812.7056, 3842.1536, 3822.2544, 3811.0898, 3822.4165,
       3779.1003, 3739.9888, 3734.0974, 3737.431 , 3789.6628, 3823.1306,
       3833.81  , 3861.3923, 3891.199 , 3905.0046, 3893.2144, 3885.0884,
       3907.5442, 3922.4858, 3898.9775, 3886.9875, 3904.1694, 3872.5398,
       3830.536 , 3843.3293, 3833.0637, 3809.371 , 3834.1235, 3858.669 ,
       3836.5774, 3756.4077, 3748.7395, 3801.937 , 3849.7766, 3882.9097,
       3914.886 , 3912.7964, 3925.2373, 3952.0027, 

# BPNN

In [4]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,1])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-19.058626011466448, p_value=1.361635667538202e-63)
crit：MAE， mdm_return(MDM=-24.201509616812423, p_value=1.2039556077591686e-90)
crit：MAPE， mdm_return(MDM=-24.66682990939473, p_value=4.07164151356023e-93)


# SVR

In [5]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,2])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-17.987926467024188, p_value=4.124464457176294e-58)
crit：MAE， mdm_return(MDM=-22.94190698851694, p_value=5.822048233901907e-84)
crit：MAPE， mdm_return(MDM=-23.474641720872675, p_value=8.697584611677207e-87)


# GRU

In [6]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,3])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.381352742644488, p_value=4.535205208159707e-50)
crit：MAE， mdm_return(MDM=-20.23307593996409, p_value=1.0901830269502438e-69)
crit：MAPE， mdm_return(MDM=-20.41165841578588, p_value=1.2713328174726986e-70)


# LSTM

In [7]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,4])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-17.42941538804793, p_value=2.745459103052615e-55)
crit：MAE， mdm_return(MDM=-21.87986475383661, p_value=2.427702159478423e-78)
crit：MAPE， mdm_return(MDM=-22.01196388418348, p_value=4.868867207941949e-79)


# Transformer

In [8]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,5])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-18.049768279388246, p_value=2.000472666526027e-58)
crit：MAE， mdm_return(MDM=-22.460337754602502, p_value=2.0706763884251047e-81)
crit：MAPE， mdm_return(MDM=-22.846651444122237, p_value=1.8619266680844406e-83)


# Informer

In [9]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,6])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-12.860004104844842, p_value=1.4103088082268555e-33)
crit：MAE， mdm_return(MDM=-19.036967151320475, p_value=1.7609085578228e-63)
crit：MAPE， mdm_return(MDM=-18.615067892782992, p_value=2.5993582603388823e-61)


# TCN

In [10]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,7])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-13.415811189173809, p_value=4.746071106089818e-36)
crit：MAE， mdm_return(MDM=-19.79933773739347, p_value=1.985088578313547e-67)
crit：MAPE， mdm_return(MDM=-19.25181971845046, p_value=1.3696686904825843e-64)


# BiLSTM

In [11]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,8])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.348644536488155, p_value=6.570247656370332e-50)
crit：MAE， mdm_return(MDM=-20.022045977286844, p_value=1.3752433175906765e-68)
crit：MAPE， mdm_return(MDM=-20.177111668930724, p_value=2.1362703308371179e-69)


# CNN-BiLSTM-SAM

In [12]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,9])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.295329361235236, p_value=4.50202881622819e-40)
crit：MAE， mdm_return(MDM=-20.281208195472185, p_value=6.110996136817069e-70)
crit：MAPE， mdm_return(MDM=-20.18793387888678, p_value=1.875734137300643e-69)


# ICEEMDAN-SVR

In [13]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,10])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-18.08637796311551, p_value=1.3030555955410316e-58)
crit：MAE， mdm_return(MDM=-21.882369562921244, p_value=2.3548793053673683e-78)
crit：MAPE， mdm_return(MDM=-22.033102550218313, p_value=3.764706202946814e-79)


# CEEMDAN-LSTM

In [14]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,11])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-14.183759771693092, p_value=1.4816009344636926e-39)
crit：MAE， mdm_return(MDM=-19.347650723817374, p_value=4.3751472394392165e-65)
crit：MAPE， mdm_return(MDM=-18.806888192278972, p_value=2.6924816871792583e-62)


# ICEEMDAN-BiGRU

In [15]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,12])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-13.29539089550798, p_value=1.6477255293843492e-35)
crit：MAE， mdm_return(MDM=-15.051570589358686, p_value=1.251114350424528e-43)
crit：MAPE， mdm_return(MDM=-14.837561018146818, p_value=1.2945935625991675e-42)


# VMD-BiLSTM

In [16]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,13])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-10.81876910016486, p_value=4.98071477127482e-25)
crit：MAE， mdm_return(MDM=-13.37596729669938, p_value=7.169238144093355e-36)
crit：MAPE， mdm_return(MDM=-12.885630717531901, p_value=1.0877966990544393e-33)


# BiLSTM-SAM-TCN

In [17]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,14])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-12.573824658777735, p_value=2.5122571564156197e-32)
crit：MAE， mdm_return(MDM=-17.864177666204753, p_value=1.7507479175539795e-57)
crit：MAPE， mdm_return(MDM=-17.70426673028158, p_value=1.1288622499425939e-56)


# ICEEMDAN-BiLSTM-SAM-TCN

In [18]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,15])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-7.774480713377896, p_value=3.341449045027959e-14)
crit：MAE， mdm_return(MDM=-10.173153049775, p_value=1.5955733687133806e-22)
crit：MAPE， mdm_return(MDM=-9.739920201360391, p_value=6.681066048619891e-21)


# ICEEMDAN-VMD(PSO)-SAM-TCN

In [19]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,16])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.605466686818636, p_value=3.2332595456020316e-28)
crit：MAE， mdm_return(MDM=-13.43923534612789, p_value=3.7229586242551773e-36)
crit：MAPE， mdm_return(MDM=-13.224471951997005, p_value=3.419940064033406e-35)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM

In [20]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,17])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-7.968815088039682, p_value=8.180137680365665e-15)
crit：MAE， mdm_return(MDM=-10.747299061128965, p_value=9.54357133694793e-25)
crit：MAPE， mdm_return(MDM=-10.320009540125241, p_value=4.386379229294415e-23)


# ICEEMDAN-VMD(PSO)-BiLSTM-TCN

In [21]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,18])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-16.252162128902704, p_value=1.9577189432470522e-49)
crit：MAE， mdm_return(MDM=-17.964213543397204, p_value=5.4422243719845326e-58)
crit：MAPE， mdm_return(MDM=-17.942186145318757, p_value=7.040202042828726e-58)


# CEEMDAN-VMD(PSO)-BiLSTM-SAM-TCN

In [22]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,19])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-8.458349216977895, p_value=2.0973727314979632e-16)
crit：MAE， mdm_return(MDM=-10.714032467295537, p_value=1.2904639944018136e-24)
crit：MAPE， mdm_return(MDM=-10.550342784389734, p_value=5.643933839698868e-24)


# ICEEMDAN-VMD(PSO)-BiLSTM-SAM-CNN

In [23]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,20])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-5.842723647124947, p_value=8.453648789695053e-09)
crit：MAE， mdm_return(MDM=-6.062540700788713, p_value=2.3786507937224206e-09)
crit：MAPE， mdm_return(MDM=-6.010758598655915, p_value=3.2180651556250068e-09)


# ICEEMDAN-VMD(PSO)-LSTM-SAM-TCN

In [24]:
pred2_lst  =read_csv('./True value of S&P 500 Low and all model predictions.csv', index_col=0,usecols=[0,21])
pred2_lst=pred2_lst.values
pred2_lst=pred2_lst.flatten()
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="RMSE")
print("crit：RMSE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAE")
print("crit：MAE，", rt)
rt = mdm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MAPE")
print("crit：MAPE，", rt)

crit：RMSE， mdm_return(MDM=-11.572140254731758, p_value=4.441473576132865e-28)
crit：MAE， mdm_return(MDM=-13.86451673698362, p_value=4.3650053867795364e-38)
crit：MAPE， mdm_return(MDM=-13.615154352727796, p_value=5.969164699141978e-37)
